In [26]:
import pandas
from plotly import graph_objects as go
from pathlib import Path
import os
if not Path("jw_utils").exists():
    !git clone https://github.com/JonWinkelman/jw_utils
from jw_utils import parse_fasta as pfa
from jw_utils import parse_gff as pgf

In [2]:
comp_genome_1 = Path("./data/3FF69J_results/3FF69J_1_1.1/3FF69J_1_1.1_1_reference.fasta")
comp_genome_2 = Path("./data/3FF69J_results/3FF69J_2_2.2/3FF69J_2_2.2_1_reference.fasta")
ref_genome    = Path("./references/GCA_000196175.1_ASM19617v1_genomic.fna")

In [32]:

def make_dnadiff_comp(ref_genome, comp_genome, outdir):
    outdir.mkdir(exist_ok=True, parents=True)
    comp = comp_genome.absolute()
    ref = ref_genome.absolute()
    working_dir = os.getcwd()
    os.chdir(outdir)
    !dnadiff $ref $comp
    !mummerplot --png  --layout --fat out.delta
    os.chdir(working_dir)


outdir_1 = Path(f'./results/dnadiff/{comp_genome_1.stem}')
make_dnadiff_comp(ref_genome, comp_genome_1, outdir_1)
outdir_2 = Path(f'./results/dnadiff/{comp_genome_2.stem}')
make_dnadiff_comp(ref_genome, comp_genome_2, outdir_2)

Building alignments
1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 3782951
# construct suffix tree for sequence of length 3782951
# (maximum reference length is 2305843009213693948)
# (maximum query length is 18446744073709551615)
# process 37829 characters per dot
#....................................................................................................
# CONSTRUCTIONTIME /Users/jonathanwinkelman/miniconda3/envs/mummer/opt/mummer-3.23/mummer out.ntref 0.61
# reading input file "/Users/jonathanwinkelman/Trestle/Mukherjee_lab/20260105_analiml_bdello_genome_comp/mummer_genome_comparison/data/3FF69J_results/3FF69J_1_1.1/3FF69J_1_1.1_1_reference.fasta" of length 3789023
# matching query-file "/Users/jonathanwinkelman/Trestle/Mukherjee_lab/20260105_analiml_bdello_genome_comp/mummer_genome_comparison/data/3FF69J_results/3FF69J_1_1.1/3FF69J_1_1.1_1_reference.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /Users/jonat

| Col # | Meaning                           |
| ----- | --------------------------------- |
| 1     | `ref_pos` — position in reference |
| 2     | `ref_base`                        |
| 3     | `query_base`                      |
| 4     | `query_pos`                       |
| 5     | `ref_pos_in_aln`                  |
| 6     | `query_pos_in_aln`                |
| 7     | `ref_contig_len`                  |
| 8     | `query_contig_len`                |
| 9     | `ref_strand`                      |
| 10    | `query_strand`                    |
| 11    | `ref_contig`                      |
| 12    | `query_contig`                    |  

. in ref_base → insertion in query  
. in query_base → deletion in query  
query_strand = -1 → alignment is reversed (inversion or opposite orientation)  
Multiple rows with same ref_pos → multi-base indels (expected)  


In [25]:
import pandas as pd
def make_snp_df(snps_fp):
    snps_df = pd.read_csv(
        snps_fp,
        sep="\t",
        header=None
    )
    
    snps_df.columns = [
        "ref_pos",
        "ref_base",
        "query_base",
        "query_pos",
        "ref_pos_aln",
        "query_pos_aln",
        "ref_contig_len",
        "query_contig_len",
        "ref_strand",
        "query_strand",
        "ref_contig",
        "query_contig",
    ]
    return snps_df
snps_fp1 = outdir_1 / 'out.snps'
snps_df1  = make_snp_df(snps_fp1)
snps_df1.to_csv(outdir_1 /  'out.snps.csv')

snps_fp2 = outdir_2 / 'out.snps'
snps_df2  = make_snp_df(snps_fp2)
snps_df2.to_csv(outdir_2 /  'out.snps.csv')

In [11]:
ref_genome_d = pfa.get_seq_dict(ref_genome)

In [39]:
path_to_gff = Path('./references/genomic.gff')
annot_df = pgf.make_simple_annot_df(path_to_gff, start_end=True, contig=True)
annot_df.head()

,protein_ID,common_name,product,start,end,strand,contig
gene_ID,,,,,,,
gene-Bd0001,cds-CAE77685.1,dnaA,chromosomal replication initiator protein dnaA,1,1416,+,BX842601.2
gene-Bd0002,cds-CAE77686.1,dnaN,DNA polymerase III%2C beta subunit,1649,2755,+,BX842601.2
gene-Bd0003,cds-CAE77687.1,Bd0003,DNA repair and genetic recombination protein,2755,3879,+,BX842601.2
gene-Bd0004,cds-CAE77688.1,Bd0004,DNA gyrase subunit B,3966,6386,+,BX842601.2
gene-Bd0005,cds-CAE77689.1,gyrA,DNA gyrase subunit A,6410,8893,+,BX842601.2


In [111]:
import bisect
def map_snps_to_genes(snps_df, annot_df): 
    
    colnames  = [
        "protein_ID",
        "common_name",
        "product",
        "start",
        "end",
        "strand",
        "contig",
        'location', 
        'snp_id',
        'snp_position'
        ]
    cols_d = {c:[] for c in colnames}
    starts = list(annot_df['start'])
    ends = list(annot_df['end'])
    
    
    
    for ind, row in snps_df.iterrows():
        start_index = bisect.bisect_left(starts, row['ref_pos'])
        end_index   = bisect.bisect_left(ends, row['ref_pos'])
        s = annot_df.iloc[end_index]
        cols_d['snp_id'].append(ind)
        cols_d['snp_position'].append(row['ref_pos'])
        for col in colnames:
            if col in s.index:
                cols_d[col].append(s[col])
            elif col == 'location':
                if start_index-1 == end_index: 
                    location = 'within_gene'
                    cols_d[col].append(location)
                else:
                    location = 'intergenic'
                    cols_d[col].append(location)
    return snps_df.join(pd.DataFrame(cols_d).set_index('snp_id'))           

snps_annot_df1 = map_snps_to_genes(snps_df1, annot_df)
snps_annot_df2 = map_snps_to_genes(snps_df2, annot_df)

In [113]:
snps_annot_df1.to_csv(outdir_1 / 'out.snps_annot.csv')
snps_annot_df2.to_csv(outdir_2 / 'out.snps_annot.csv')